<a href="https://www.kaggle.com/code/diwakarsehgal/ai-powered-agricultural-optimization?scriptVersionId=236079494" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# ✨ AI-Powered Agricultural Optimization ✨

# 🌾🌾🌾
Objective - Gen AI Intensive Course Capstone 2025Q1

This notebook demonstrates an AI assistant designed to help farmers make informed decisions about crop selection. By leveraging farmer profile data and market research data stored in a database, the AI uses generative models and database tools to analyze relevant factors and provide recommendations for optimal crops. 🌱📊🤖

## Video Explaination

[YOUTUBE](https://www.youtube.com/watch?v=ruBfTk3kz8E)

## Blog Post

[MEDIUM](https://medium.com/@diwakarsehgal02/gen-ai-intensive-course-capstone-2025q1-c2cff0e65736)

## Kaggle Notebook Link

[KAGGLE](https://www.kaggle.com/code/diwakarsehgal/ai-powered-agricultural-optimization/notebook#Tool-Definition-and-Testing)

## 🎯 Problem Statement and Gen AI Solution

This project addresses the challenge farmers face in selecting the most suitable crops based on various factors like soil conditions, preferences, and market trends. We demonstrate how Generative AI, specifically a large language model equipped with database interaction tools, can process relevant data and provide tailored recommendations to optimize agricultural outcomes.

## 💻 Code Implementation and Explanation

The following sections of the notebook detail the step-by-step implementation of the AI-powered solution. We show how to set up the environment, prepare the data, define tools for database interaction, configure the Gen AI model with these tools, and simulate a conversation where the AI utilizes the tools to analyze data and generate crop recommendations. Each code section is accompanied by Markdown explanations for clarity.

## ✨ Demonstrated Gen AI Capabilities

* **Function Calling:** The model calls predefined Python functions (`list_tables`, `describe_table`, `execute_query`) to interact with the database and retrieve information.
* **Long context window:** The model maintains the history of the conversation, including user inputs and tool outputs, to provide relevant responses and make decisions across multiple turns.
* **Agents:** The AI acts as an intelligent agent, interpreting instructions, deciding which tools to use, and synthesizing information to achieve the goal of recommending optimal crops.

## 🔧 Environment Setup and Library Imports

This section handles the initial setup of the environment by installing necessary packages and importing the required Python libraries. This includes libraries for data manipulation (`pandas`), database interaction (`sqlite3`), file system operations (`os`), and Google's Generative AI library (`google-genai`). ⚙️📦

In [1]:
!pip uninstall -qqy jupyterlab  # Remove unused conflicting packages
!pip install -U -q "google-genai==1.7.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 4.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyterlab-lsp 3.10.2 requires jupyterlab<4.0.0a0,>=3.1.0, which is not installed.


In [2]:
from google import genai
from google.genai import types
import pandas as pd
import sqlite3
import os

genai.__version__

'1.7.0'

In [3]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
print("Google API Key configured successfully.")

Google API Key configured successfully.


In [4]:
# Define a retry policy. The model might make multiple consecutive calls automatically
# for a complex query, this ensures the client retries if it hits quota limits.
from google.api_core import retry

is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

if not hasattr(genai.models.Models.generate_content, '__wrapped__'):
  genai.models.Models.generate_content = retry.Retry(
      predicate=is_retriable)(genai.models.Models.generate_content)

## 📂 Data Preparation and Database Initialization

Here, we define the file paths for the datasets and create a function to initialize an SQLite database. This involves loading data from CSV files into the database, creating tables, and handling potential errors during the process. 📑➡️🗄️

In [5]:
# Define file paths (adjust if your Kaggle input structure differs)
farmer_csv_path = '/kaggle/input/farmerdataset/farmer_advisor_dataset.csv'
market_csv_path = '/kaggle/input/farmerdataset/market_researcher_dataset.csv'
db_file = "agriculture_optimization.db"

print(f"Database file: {db_file}")
print(f"Farmer data path: {farmer_csv_path}")
print(f"Market data path: {market_csv_path}")

Database file: agriculture_optimization.db
Farmer data path: /kaggle/input/farmerdataset/farmer_advisor_dataset.csv
Market data path: /kaggle/input/farmerdataset/market_researcher_dataset.csv


In [6]:
# Function to create tables and load data from CSV
def initialize_database(db_path, farmer_csv, market_csv):
    """Creates SQLite tables and loads data from CSV files."""
    conn = None # Initialize conn to None
    try:
        # Delete existing database file to start fresh
        if os.path.exists(db_path):
            os.remove(db_path)
            print(f"Existing database '{db_path}' removed.")

        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        print(f"Database '{db_path}' created/connected.")

        # Load and process Farmer Advisor data
        if os.path.exists(farmer_csv):
            farmer_df = pd.read_csv(farmer_csv)
            # Basic schema assumption (modify based on actual CSV columns)
            # For simplicity, converting complex types like preferences to TEXT
            farmer_df.columns = farmer_df.columns.str.replace('[^A-Za-z0-9_]+', '', regex=True) # Sanitize column names
            farmer_df.to_sql('farmer_profiles', conn, if_exists='replace', index=False)
            print(f"Loaded data from '{farmer_csv}' into 'farmer_profiles' table.")
            # Display schema
            cursor.execute("PRAGMA table_info(farmer_profiles);")
            print("Schema for 'farmer_profiles':", cursor.fetchall())
        else:
            print(f"Warning: Farmer CSV not found at {farmer_csv}. 'farmer_profiles' table not created.")


        # Load and process Market Researcher data
        if os.path.exists(market_csv):
            market_df = pd.read_csv(market_csv)
            # Basic schema assumption (modify based on actual CSV columns)
            market_df.columns = market_df.columns.str.replace('[^A-Za-z0-9_]+', '', regex=True) # Sanitize column names
            market_df.to_sql('market_data', conn, if_exists='replace', index=False)
            print(f"Loaded data from '{market_csv}' into 'market_data' table.")
             # Display schema
            cursor.execute("PRAGMA table_info(market_data);")
            print("Schema for 'market_data':", cursor.fetchall())
        else:
             print(f"Warning: Market CSV not found at {market_csv}. 'market_data' table not created.")


        conn.commit()
        print("Database initialized successfully.")

    except pd.errors.EmptyDataError as e:
        print(f"Error loading CSV: {e}. The file might be empty or corrupted.")
    except sqlite3.Error as e:
        print(f"SQLite error during initialization: {e}")
    except Exception as e:
        print(f"An unexpected error occurred during database initialization: {e}")
    finally:
        if conn:
            conn.close()
            print("Database connection closed.")

# Initialize the database
initialize_database(db_file, farmer_csv_path, market_csv_path)

Database 'agriculture_optimization.db' created/connected.
Loaded data from '/kaggle/input/farmerdataset/farmer_advisor_dataset.csv' into 'farmer_profiles' table.
Schema for 'farmer_profiles': [(0, 'Farm_ID', 'INTEGER', 0, None, 0), (1, 'Soil_pH', 'REAL', 0, None, 0), (2, 'Soil_Moisture', 'REAL', 0, None, 0), (3, 'Temperature_C', 'REAL', 0, None, 0), (4, 'Rainfall_mm', 'REAL', 0, None, 0), (5, 'Crop_Type', 'TEXT', 0, None, 0), (6, 'Fertilizer_Usage_kg', 'REAL', 0, None, 0), (7, 'Pesticide_Usage_kg', 'REAL', 0, None, 0), (8, 'Crop_Yield_ton', 'REAL', 0, None, 0), (9, 'Sustainability_Score', 'REAL', 0, None, 0)]
Loaded data from '/kaggle/input/farmerdataset/market_researcher_dataset.csv' into 'market_data' table.
Schema for 'market_data': [(0, 'Market_ID', 'INTEGER', 0, None, 0), (1, 'Product', 'TEXT', 0, None, 0), (2, 'Market_Price_per_ton', 'REAL', 0, None, 0), (3, 'Demand_Index', 'REAL', 0, None, 0), (4, 'Supply_Index', 'REAL', 0, None, 0), (5, 'Competitor_Price_per_ton', 'REAL', 0, No

## 🛠️ Database Interaction Tools

These functions serve as the interface for the AI model to interact with the SQLite database. They provide capabilities to list available tables, inspect table schemas, and execute read-only SQL queries. 🔍📚

In [7]:
# Establish a persistent connection for the tools
# Ensure the database file exists after initialization
if os.path.exists(db_file):
    db_conn = sqlite3.connect(db_file)
    print(f"Connected to database '{db_file}' for function calls.")
else:
    print(f"Error: Database file '{db_file}' not found. Please run Cell 2 first.")
    db_conn = None # Ensure db_conn is None if connection fails

# --- Database Interaction Functions (Adapted from PDF [cite: 1]) ---

def list_tables() -> list[str]:
    """Retrieve the names of all tables in the database."""
    print('\n>> DB CALL: list_tables()')
    if not db_conn:
        return ["Error: Database connection not established."]
    try:
        cursor = db_conn.cursor()
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = [t[0] for t in cursor.fetchall() if t[0] not in ('sqlite_sequence',)] # Exclude internal tables
        print(f"<< DB RESULT: {tables}")
        return tables
    except sqlite3.Error as e:
        print(f"<< DB ERROR: {e}")
        return [f"Error listing tables: {e}"]

def describe_table(table_name: str) -> list[tuple[str, str]]:
    """Look up the schema (column names and types) for a specific table.
    Args:
        table_name: The name of the table to describe.
    Returns:
        List of columns, where each entry is a tuple of (column_name, column_type).
    """
    print(f'\n>> DB CALL: describe_table(table_name="{table_name}")')
    if not db_conn:
        return [("Error", "Database connection not established.")]
    if not table_name or not isinstance(table_name, str):
        print("<< DB ERROR: Invalid table name provided.")
        return [("Error", "Invalid table name provided.")]

    try:
        cursor = db_conn.cursor()
        # Parameterized query to prevent SQL injection, even for PRAGMA
        cursor.execute(f"PRAGMA table_info({table_name});") # PRAGMA doesn't support placeholders directly
        schema = cursor.fetchall()
        # schema format: [(index, name, type, notnull, default_value, pk), ...]
        columns = [(col[1], col[2]) for col in schema] # Extract (name, type)
        print(f"<< DB RESULT for {table_name}: {columns}")
        return columns
    except sqlite3.Error as e:
        print(f"<< DB ERROR describing {table_name}: {e}")
        return [("Error", f"Could not describe table {table_name}: {e}")]

def execute_query(sql: str) -> list[list[str]]:
    """Execute a *read-only* SQL SELECT statement, returning the results.
       IMPORTANT: This function is restricted to SELECT queries for safety.
    Args:
        sql: The SQL SELECT query to execute.
    Returns:
        A list of rows, where each row is a list of strings.
    """
    print(f'\n>> DB CALL: execute_query(sql="{sql}")')
    if not db_conn:
         return [["Error: Database connection not established."]]
    if not sql or not isinstance(sql, str) or not sql.strip().upper().startswith("SELECT"):
        print("<< DB ERROR: Invalid or non-SELECT query provided.")
        return [["Error: Only SELECT queries are allowed."]]

    try:
        cursor = db_conn.cursor()
        cursor.execute(sql)
        results = cursor.fetchall()
        # Convert results to list of lists of strings for consistency
        results_str = [[str(item) for item in row] for row in results]
        print(f"<< DB RESULT (first 5 rows): {results_str[:5]}")
        if len(results_str) > 5:
            print(f"... ({len(results_str) - 5} more rows)")
        return results_str
    except sqlite3.Error as e:
        print(f"<< DB ERROR executing query: {e}")
        return [[f"Error executing query: {e}"]]

# --- List of tools for the Gemini Model ---
db_tools = [list_tables, describe_table, execute_query]

# --- Test the functions (optional) ---
print("\n--- Testing Database Functions ---")
tables = list_tables()
if tables and not "Error" in tables[0]:
    for table in tables:
        describe_table(table)
    # Example query (adjust based on actual columns in market_data)
    if 'market_data' in tables:
        execute_query("SELECT * FROM market_data LIMIT 2;")
    if 'farmer_profiles' in tables:
         execute_query("SELECT * FROM farmer_profiles LIMIT 2;")
else:
    print("Skipping further tests due to issues listing tables.")
print("--- Finished Testing ---")

Connected to database 'agriculture_optimization.db' for function calls.

--- Testing Database Functions ---

>> DB CALL: list_tables()
<< DB RESULT: ['farmer_profiles', 'market_data']

>> DB CALL: describe_table(table_name="farmer_profiles")
<< DB RESULT for farmer_profiles: [('Farm_ID', 'INTEGER'), ('Soil_pH', 'REAL'), ('Soil_Moisture', 'REAL'), ('Temperature_C', 'REAL'), ('Rainfall_mm', 'REAL'), ('Crop_Type', 'TEXT'), ('Fertilizer_Usage_kg', 'REAL'), ('Pesticide_Usage_kg', 'REAL'), ('Crop_Yield_ton', 'REAL'), ('Sustainability_Score', 'REAL')]

>> DB CALL: describe_table(table_name="market_data")
<< DB RESULT for market_data: [('Market_ID', 'INTEGER'), ('Product', 'TEXT'), ('Market_Price_per_ton', 'REAL'), ('Demand_Index', 'REAL'), ('Supply_Index', 'REAL'), ('Competitor_Price_per_ton', 'REAL'), ('Economic_Indicator', 'REAL'), ('Weather_Impact_Score', 'REAL'), ('Seasonal_Factor', 'TEXT'), ('Consumer_Trend_Index', 'REAL')]

>> DB CALL: execute_query(sql="SELECT * FROM market_data LIMIT 

# Tool Definition and Testing

This section defines the list of database tools that will be provided to the Generative AI model. It also includes an optional section to test these tools to ensure they are functioning correctly.

# AI Assistant Configuration

Here, we configure the Generative AI assistant. This includes defining the system instruction that guides the AI's behavior and initializing the chat model with the defined tools and configuration.

In [8]:
# These are the Python functions defined above.
db_tools = [list_tables, describe_table, execute_query]

instruction = f"""You are an AI assistant designed to help farmers choose the best crops to plant based on sustainability and profitability. You have access to two primary data sources in an SQLite database:
1.  'farmer_profiles': Contains information about farmer's land, preferences, and goals (use describe_table('farmer_profiles') to see exact columns).
2.  'market_data': Contains market trends, pricing, demand, and sustainability metrics for various crops (use describe_table('market_data') to see exact columns).

Your goal is to recommend optimal crops. To do this, you MUST first understand the farmer's specific situation and the available data.

Follow these steps:
1.  **Inspect Tables:** Use `list_tables()` and `describe_table()` for both 'farmer_profiles' and 'market_data' to understand their schemas (columns and data types). **Do this first.**
2.  **Ask Clarifying Questions:** Based *only* on the columns present in the 'farmer_profiles' and 'market_data' tables, ask the user clear, specific questions to gather necessary information for filtering. For example, if 'farmer_profiles' has a 'region' column, ask "What region is your farm located in?". If 'market_data' has 'water_usage_index', and the farmer wants sustainable options, you might ask about water availability later. **Do NOT ask for information not present as columns in the database tables.** Start by asking essential filtering questions (like region, soil type, budget if available in the schema).
3.  **Formulate Queries:** Once you have gathered sufficient information from the user, use `execute_query()` with SQL SELECT statements to filter the 'market_data' table. Your queries should join or filter based on the user's answers and the columns identified in step 1 (e.g., filter by region, soil compatibility, budget constraints, market demand, sustainability metrics like water usage or carbon footprint).
4.  **Synthesize Recommendation:** Analyze the results from your SQL query. Present the top 1-3 crop choices to the user, explaining *why* they are suitable based on the data (e.g., "Crop X is recommended because it matches your region, has high demand forecast, and aligns with your preference for low water usage.").
5.  **Tool Usage:** Only use the provided tools (`list_tables`, `describe_table`, `execute_query`). Only use `execute_query` for SELECT statements. Be precise with table and column names as discovered by `describe_table`.

Start by inspecting the tables and then asking the user the most critical initial questions based on the table schemas.
"""

client = genai.Client(api_key=GOOGLE_API_KEY)

# Start a chat with automatic function calling enabled.
chat = client.chats.create(
    model="gemini-2.0-flash",
    config=types.GenerateContentConfig(
        system_instruction=instruction,
        tools=db_tools,
    ),
)

In [9]:
# Send an initial message to trigger the AI's process (inspecting tables)
initial_prompt = "Please start by analyzing the available data tables."
print(f"\nUser: {initial_prompt}")

try:
    response = chat.send_message(initial_prompt)
    print(f"\nAI Assistant: {response.text}")
except Exception as e:
     print(f"\nError during initial chat interaction: {e}")


User: Please start by analyzing the available data tables.

>> DB CALL: list_tables()
<< DB RESULT: ['farmer_profiles', 'market_data']

>> DB CALL: describe_table(table_name="farmer_profiles")
<< DB RESULT for farmer_profiles: [('Farm_ID', 'INTEGER'), ('Soil_pH', 'REAL'), ('Soil_Moisture', 'REAL'), ('Temperature_C', 'REAL'), ('Rainfall_mm', 'REAL'), ('Crop_Type', 'TEXT'), ('Fertilizer_Usage_kg', 'REAL'), ('Pesticide_Usage_kg', 'REAL'), ('Crop_Yield_ton', 'REAL'), ('Sustainability_Score', 'REAL')]

>> DB CALL: describe_table(table_name="market_data")
<< DB RESULT for market_data: [('Market_ID', 'INTEGER'), ('Product', 'TEXT'), ('Market_Price_per_ton', 'REAL'), ('Demand_Index', 'REAL'), ('Supply_Index', 'REAL'), ('Competitor_Price_per_ton', 'REAL'), ('Economic_Indicator', 'REAL'), ('Weather_Impact_Score', 'REAL'), ('Seasonal_Factor', 'TEXT'), ('Consumer_Trend_Index', 'REAL')]

AI Assistant: Okay, I have analyzed the schemas for both tables. Now I need to ask some questions to gather inf

# AI Conversation Simulation

This section simulates a conversation with the configured AI assistant. It starts with an initial prompt to trigger the AI's analysis and then runs through a series of pre-defined user messages to demonstrate the interaction flow.

In [10]:
# Pre-defined messages simulating a user conversation
user_messages = [
    "hello",
    "soil_ph is 7, moisture is 50, temparature is 27.",
    "200",
    "High",
    "can you consult the farmer database to see what crops might be compatible for me?",
    "can you compare the market result and farmer result to see which crop is most suitable ?",
    "Thank you",
    "quit"
]

# Run through the conversation automatically
for user_input in user_messages:
    print(f"\nYou: {user_input}")
    
    if user_input.lower() == "quit":
        print("\nAI Assistant: Goodbye! Hope this was helpful.")
        break

    try:
        response = chat.send_message(user_input)
        if response.text:
            print(f"\nAI Assistant: {response.text}")
        else:
            print("\nAI Assistant: (Processing information...)")
    except Exception as e:
        print(f"\nAn error occurred during the chat: {e}")

# Close DB connection if exists
if db_conn:
    db_conn.close()
    print("\nDatabase connection closed.")


You: hello

AI Assistant: I need some information from you to provide the best crop recommendation. Please provide the following information about your farm:

1.  What is the Soil_pH level on your farm?
2.  What is the Soil_Moisture level on your farm?
3.  What is the average Temperature_C on your farm?
4.  What is the average Rainfall_mm on your farm?


You: soil_ph is 7, moisture is 50, temparature is 27.

AI Assistant: Okay, I need one more piece of information about your farm's average Rainfall in mm.


You: 200

>> DB CALL: execute_query(sql="SELECT Product, Market_Price_per_ton, Demand_Index FROM market_data ORDER BY Demand_Index DESC LIMIT 3")
<< DB RESULT (first 5 rows): [['Corn', '257.80044917411885', '199.9974490945231'], ['Soybean', '141.79044396218066', '199.98162439309365'], ['Corn', '374.4731086521783', '199.97382614310328']]

AI Assistant: Based on the market data, the top 3 crops with the highest demand are Corn (with prices of 257.80 and 374.47 per ton) and Soybean (w

# The END
## *Thank You*
### Team Name = Solis
### - Diwakar Sehgal

## 📚 References

Here are the references used in this project:

* **Dataset:** Dataset from the Geeksforgeeks Gen AI Hackathon. [https://www.geeksforgeeks.org/hack-a-thon/gen-ai-hackathon](https://www.geeksforgeeks.org/hack-a-thon/gen-ai-hackathon)
* **Capstone Competition:** Gen AI Intensive Course Capstone 2025Q1: Addison Howard, Brenda Flynn, Myles O'Neill, Nate, and Polong Lin, 2025. Kaggle, Available at: [https://kaggle.com/competitions/gen-ai-intensive-course-capstone-2025q1](https://kaggle.com/competitions/gen-ai-intensive-course-capstone-2025q1).
* **Foundational Large Language Models & Text Generation:** Mohammadamin Barektain, Anant Nawalgaria, Daniel J. Mankowitz, Majd Al Merey, Yaniv Leviathan, Massimo Mascaro, Matan Kalman, Elena Buchatskaya, Aliaksei Severyn, Irina Sigler, and Antonio Gulli.
* **Prompt Engineering:** Lee Boonstra.
* **Evaluating Large Language Models—Principles, Approaches, and Applications:** Irina Sigler and Yuan (Emily) Xue. Presented in the NeurIPS Tutorial.
* **Embeddings & Vector Stores:** Anant Nawalgaria, Xiaoqi Ren, and Charles Sugnet.
* **Agents:** Julia Wiesinger, Patrick Marlow and Vladimir Vuskovic.
* **Agents Companion:** Antonio Gulli, Lavi Nigam, Julia Wiesinger, Vladimir Vuskovic, Irina Sigler, Ivan Nardini, Nicolas Stroppa, Sokratis Kartakis, Narek Saribekyan, and Alan Bount.
* **Solving Domain-Specific Problems Using LLMs:** Christopher Semturs, Shekoofeh Azizi, Scott Coull, Umesh Shankar and Wieland Holfelder.
* **Operationalizing Generative AI on Vertex AI using MLOps:** Anant Nawalgaria, Gabriela Hernandez Larios, Elia Secchi, Mike Styer, Christos Aniftos, Onofrio Petragallo, and Sokratis Kartakis.